In [ ]:
import requests
import lxml.html
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from dateutil.relativedelta import relativedelta
import urllib.request

In [ ]:
obs_stations = pd.read_csv("obs_stations.csv", encoding='shift_jis',header=1)
obs_stations = obs_stations.query('ed_y > 2019')
obs_stations = obs_stations.query('気温 == "Y"')
obs_stations = obs_stations.reset_index(drop=True)

In [ ]:
obs_stations.head(10)

In [ ]:
#2020年用コード
start_time = datetime.date(2020, 1, 1)
end_time   = datetime.date(2020, 12, 31)

now=time.strftime('%Y/%m/%d %H:%M:%S')
print(now)
Temp_df = pd.DataFrame(index=[], columns=['地点コード','地点','地点カナ','都道府県番号','区分','月','緯度','経度','降水量合計','平均気温','平均湿度','平均風速','日照時間合計','降雪日数'])
Temp_df_copy1 = Temp_df.copy()
Temp_df_copy2 = Temp_df.copy()
for i in range(len(obs_stations)):
    Temp_df_copy2 = Temp_df_copy1.copy()
    month = start_time
    while month < end_time:
        Temp_df_copy2.loc[month,'地点コード']=obs_stations['地点コード'][i]
        Temp_df_copy2.loc[month,'地点']=obs_stations['地点'][i]
        Temp_df_copy2.loc[month,'地点カナ']=obs_stations['地点カナ'][i]
        Temp_df_copy2.loc[month,'都道府県番号']=obs_stations['府県番号'][i]
        Temp_df_copy2.loc[month,'区分']=obs_stations['区分'][i]
        Temp_df_copy2.loc[month,'月']=month.month
        Temp_df_copy2.loc[month,'緯度']=obs_stations['緯度'][i]
        Temp_df_copy2.loc[month,'経度']=obs_stations['経度'][i]
        month += relativedelta(months=1)
    Temp_df = pd.concat([Temp_df,Temp_df_copy2], axis=0,ignore_index=True)  

def str2float(weather_data):
    try:
        return float(weather_data)
    except:
        return -100

def safe_split_string(element, default='NaN'):
    if element is not None and element.string is not None:
        return element.string.split(' ')[0]
    else:
        return default

# 各観測所のデータを取得
Temp_df_result = Temp_df.copy()
num = 0
for index, value in obs_stations.iterrows():
    date_month = start_time

    url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_%s1.php?prec_no=%d&block_no=%04d&year=%d&month=&day=&view=" \
        % (str.lower(obs_stations['区分'].loc[index]), value['府県番号'], value['地点コード'], date_month.year)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find("table", {"class": "data2_s"})

    if trs is None:
        continue

    print(f"\r{index, obs_stations['地点'].loc[index], obs_stations['地点コード'].loc[index], date_month.year}", end="")
    if obs_stations['区分'].loc[index] == 'S':
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result.loc[num, '降水量合計'] = str2float(safe_split_string(tds[3]))
            Temp_df_result.loc[num, '平均気温'] = str2float(safe_split_string(tds[7]))
            Temp_df_result.loc[num, '平均湿度'] = str2float(safe_split_string(tds[12]))
            Temp_df_result.loc[num, '平均風速'] = str2float(safe_split_string(tds[14]))
            Temp_df_result.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[19]))
            Temp_df_result.loc[num, '降雪日数'] = str2float(safe_split_string(tds[25]))
            date_month += relativedelta(months=1)
            num += 1
    else:
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result.loc[num, '降水量合計'] = str2float(safe_split_string(tds[1]))
            Temp_df_result.loc[num, '平均気温'] = str2float(safe_split_string(tds[5]))
            Temp_df_result.loc[num, '平均湿度'] = str2float(safe_split_string(tds[10]))
            Temp_df_result.loc[num, '平均風速'] = str2float(safe_split_string(tds[12]))
            Temp_df_result.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[17]))
            Temp_df_result.loc[num, '降雪日数'] = None
            date_month += relativedelta(months=1)
            num += 1
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    time.sleep(3)

Temp_202001_202012_by_month=Temp_df_result

Temp_df_result.to_csv("2020_by_month_data.csv", encoding="shift_jis",
                      index=False,header=True)

In [ ]:
#2019年用コード
start_time_2019 = datetime.date(2019, 1, 1)
end_time_2019   = datetime.date(2019, 12, 31)

#現在の時刻を表示
now=time.strftime('%Y/%m/%d %H:%M:%S')
print(now)

Temp_df = pd.DataFrame(index=[], columns=['地点コード','地点','地点カナ','都道府県番号','区分','月','緯度','経度','降水量合計','平均気温','平均湿度','平均風速','日照時間合計','降雪日数'])
Temp_df_2019 = Temp_df.copy()
Temp_df_copy1 = Temp_df.copy()
Temp_df_copy2 = Temp_df.copy()
    
#2019_11から2019_12までのデータフレーム
for i in range(len(obs_stations)):
    Temp_df_copy2 = Temp_df_copy1.copy()
    month = start_time_2019
    while month < end_time_2019:
        Temp_df_copy2.loc[month,'地点コード']=obs_stations['地点コード'][i]
        Temp_df_copy2.loc[month,'地点']=obs_stations['地点'][i]
        Temp_df_copy2.loc[month,'地点カナ']=obs_stations['地点カナ'][i]
        Temp_df_copy2.loc[month,'都道府県番号']=obs_stations['府県番号'][i]
        Temp_df_copy2.loc[month,'区分']=obs_stations['区分'][i]
        Temp_df_copy2.loc[month,'月']=month.month
        Temp_df_copy2.loc[month,'緯度']=obs_stations['緯度'][i]
        Temp_df_copy2.loc[month,'経度']=obs_stations['経度'][i]
        Temp_df_copy2.loc[month,'降水量合計':] = -100.0
        month += relativedelta(months=1)
    Temp_df_2019 = pd.concat([Temp_df_2019,Temp_df_copy2], axis=0,ignore_index=True) 

def str2float(weather_data):
    try:
        return float(weather_data)
    except:
        return -100

def safe_split_string(element, default='NaN'):
    if element is not None and element.string is not None:
        return element.string.split(' ')[0]
    else:
        return default

# 各観測所のデータを取得
Temp_df_result_2019 = Temp_df_2019.copy()

num = 0
for index, value in obs_stations.iterrows():
    date_month = start_time_2019

    url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_%s1.php?prec_no=%d&block_no=%04d&year=%d&month=&day=&view=" \
        % (str.lower(obs_stations['区分'].loc[index]), value['府県番号'], value['地点コード'], date_month.year)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find("table", {"class": "data2_s"})

    if trs is None:
        continue

    print(f"\r{index, obs_stations['地点'].loc[index], obs_stations['地点コード'].loc[index], date_month.year}", end="")
    if obs_stations['区分'].loc[index] == 'S':
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2019.loc[num, '降水量合計'] = str2float(safe_split_string(tds[3]))
            Temp_df_result_2019.loc[num, '平均気温'] = str2float(safe_split_string(tds[7]))
            Temp_df_result_2019.loc[num, '平均湿度'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2019.loc[num, '平均風速'] = str2float(safe_split_string(tds[14]))
            Temp_df_result_2019.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[19]))
            Temp_df_result_2019.loc[num, '降雪日数'] = str2float(safe_split_string(tds[25]))
            date_month += relativedelta(months=1)
            num += 1
    else:
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2019.loc[num, '降水量合計'] = str2float(safe_split_string(tds[1]))
            Temp_df_result_2019.loc[num, '平均気温'] = str2float(safe_split_string(tds[5]))
            Temp_df_result_2019.loc[num, '平均湿度'] = str2float(safe_split_string(tds[10]))
            Temp_df_result_2019.loc[num, '平均風速'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2019.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[17]))
            Temp_df_result_2019.loc[num, '降雪日数'] = None
            date_month += relativedelta(months=1)
            num += 1
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    time.sleep(3)

Temp_201901_201912_by_month=Temp_df_result_2019

Temp_201901_201912_by_month.to_csv("2019_by_month_data.csv", encoding="shift_jis",
                      index=False,header=True)

In [ ]:
#2021年用コード
start_time_2021 = datetime.date(2021, 1, 1)
end_time_2021   = datetime.date(2021, 12, 31)

#現在の時刻を表示
now=time.strftime('%Y/%m/%d %H:%M:%S')
print(now)

Temp_df = pd.DataFrame(index=[], columns=['地点コード','地点','地点カナ','都道府県番号','区分','月','緯度','経度','降水量合計','平均気温','平均湿度','平均風速','日照時間合計','降雪日数'])
Temp_df_2021 = Temp_df.copy()
Temp_df_copy1 = Temp_df.copy()
Temp_df_copy2 = Temp_df.copy()

#2021_1から2021_2までのデータフレーム
for i in range(len(obs_stations)):
    Temp_df_copy2 = Temp_df_copy1.copy()
    month = start_time_2021
    while month < end_time_2021:
        Temp_df_copy2.loc[month,'地点コード']=obs_stations['地点コード'][i]
        Temp_df_copy2.loc[month,'地点']=obs_stations['地点'][i]
        Temp_df_copy2.loc[month,'地点カナ']=obs_stations['地点カナ'][i]
        Temp_df_copy2.loc[month,'都道府県番号']=obs_stations['府県番号'][i]
        Temp_df_copy2.loc[month,'区分']=obs_stations['区分'][i]
        Temp_df_copy2.loc[month,'月']=month.month
        Temp_df_copy2.loc[month,'緯度']=obs_stations['緯度'][i]
        Temp_df_copy2.loc[month,'経度']=obs_stations['経度'][i]
        month += relativedelta(months=1)
    Temp_df_2021 = pd.concat([Temp_df_2021,Temp_df_copy2], axis=0,ignore_index=True) 


def str2float(weather_data):
    try:
        return float(weather_data)
    except:
        return -100

def safe_split_string(element, default='NaN'):
    if element is not None and element.string is not None:
        return element.string.split(' ')[0]
    else:
        return default

# 各観測所のデータを取得
Temp_df_result_2021 = Temp_df_2021.copy()

num = 0
for index, value in obs_stations.iterrows():
    date_month = start_time_2021

    url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_%s1.php?prec_no=%d&block_no=%04d&year=%d&month=&day=&view=" \
        % (str.lower(obs_stations['区分'].loc[index]), value['府県番号'], value['地点コード'], date_month.year)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find("table", {"class": "data2_s"})

    if trs is None:
        continue

    print(f"\r{index, obs_stations['地点'].loc[index], obs_stations['地点コード'].loc[index], date_month.year}", end="")
    if obs_stations['区分'].loc[index] == 'S':
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2021.loc[num, '降水量合計'] = str2float(safe_split_string(tds[3]))
            Temp_df_result_2021.loc[num, '平均気温'] = str2float(safe_split_string(tds[7]))
            Temp_df_result_2021.loc[num, '平均湿度'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2021.loc[num, '平均風速'] = str2float(safe_split_string(tds[14]))
            Temp_df_result_2021.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[19]))
            Temp_df_result_2021.loc[num, '降雪日数'] = str2float(safe_split_string(tds[25]))
            date_month += relativedelta(months=1)
            num += 1
    else:
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2021.loc[num, '降水量合計'] = str2float(safe_split_string(tds[1]))
            Temp_df_result_2021.loc[num, '平均気温'] = str2float(safe_split_string(tds[5]))
            Temp_df_result_2021.loc[num, '平均湿度'] = str2float(safe_split_string(tds[10]))
            Temp_df_result_2021.loc[num, '平均風速'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2021.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[17]))
            Temp_df_result_2021.loc[num, '降雪日数'] = None
            date_month += relativedelta(months=1)
            num += 1
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    time.sleep(3)

print(Temp_df_result_2021)

Temp_202101_202112_by_month=Temp_df_result_2021

Temp_202101_202112_by_month.to_csv("2021_by_month_data.csv", encoding="shift_jis",
                      index=False,header=True)

In [ ]:
data2020 = pd.read_csv("2020_by_month_data.csv",encoding='shift-jis')
data2019 = pd.read_csv("2019_by_month_data.csv",encoding='shift-jis')
data2021 = pd.read_csv("2021_by_month_data.csv",encoding='shift-jis')

#同じ地点コードの有無の確認
count = data2020.groupby('地点コード').count()
#count = data2021.groupby('地点コード').count()
#count = data2019.groupby('地点コード').count()

# 'counts' データフレームからデータ数が12よりも多い地域を抽出
more_than_12 = count[count['月'] > 12]
more_than_12.index[0]
#print(more_than_12['地点コード'])
#print(data2020[data2020['地点コード']==more_than_12.index[0]]) #都道府県コード49:山梨　50:静岡

#静岡の富士山データを落とす
data2020.drop(data2020[(data2020['地点コード'] == more_than_12.index[0]) & (data2020['都道府県番号'] == 50)].index, inplace=True)
data2021.drop(data2021[(data2021['地点コード'] == more_than_12.index[0]) & (data2021['都道府県番号'] == 50)].index, inplace=True)
data2019.drop(data2019[(data2019['地点コード'] == more_than_12.index[0]) & (data2019['都道府県番号'] == 50)].index, inplace=True)
#indexの振り直し
data2020.reset_index(drop=True, inplace=True)
data2021.reset_index(drop=True, inplace=True)
data2019.reset_index(drop=True, inplace=True)

In [ ]:
# '2020-01' から '2020-12' までの年月を生成
times_2020 = pd.date_range(start='2020-01', end='2021-01', freq='M').strftime('%Y-%m')
times_list_2020 = []

times_2021 = pd.date_range(start='2021-01', end='2022-01', freq='M').strftime('%Y-%m')
times_list_2021 = []

times_2019 = pd.date_range(start='2019-01', end='2020-01', freq='M').strftime('%Y-%m')
times_list_2019 = []

for _ in range(len(data2020['地点コード'].unique())):
    times_list_2020.extend(times_2020)
    
for _ in range(len(data2021['地点コード'].unique())):
    times_list_2021.extend(times_2021)

for _ in range(len(data2019['地点コード'].unique())):
    times_list_2019.extend(times_2019)

data2020['年月'] = times_list_2020
data2021['年月'] = times_list_2021
data2019['年月'] = times_list_2019

In [ ]:
#データの縦結合
df_con = pd.concat([data2019, data2020,data2021],ignore_index=True)
df_con

# 対象とする数値列のリスト
col = ['降水量合計', '平均気温', '平均湿度', '平均風速', '日照時間合計','降雪日数']

# 新しいデータフレームを作成し、元の識別情報をコピー
moving_ave_df = df_con[['地点', '月', '緯度', '経度', '地点コード', '地点カナ', '都道府県番号', '区分', '年月']].copy()

# 新しい変数「期間」を作成
moving_ave_df['期間'] = moving_ave_df['月'].apply(lambda x: f"{(x - 2) % 12 or 12}～{x}")

# 各数値列に対して移動平均を計算し、新しいデータフレームに追加

for column in col:
    moving_ave_df[f'{column}_移動平均'] = df_con.groupby('地点コード')[column].rolling(window=3).mean().reset_index(level=0, drop=True)

moving_ave_df

moving_ave_df.to_csv("2019-2021_by_3months.csv", encoding="UTF-8",
                      index=False,header=True)
moving_ave_df_2020 = moving_ave_df[moving_ave_df['年月'].str[:4]=='2020']

moving_ave_df_2020.to_csv("2020_by_3months.csv", encoding="UTF-8",
                      index=False,header=True)